In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from IPython.display import clear_output

from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import os

In [2]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import random

In [10]:
driver = webdriver.Chrome()

In [11]:
keyword = '김현지'
driver.get(f"https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query={keyword}")

In [15]:
# naver 주소 접속
driver.get('https://www.naver.com')

In [16]:
# 검색창에 키워드 입력
keyword = '김현지'
driver.find_element(By.ID, 'query').send_keys(keyword)

In [17]:
# 검색 버튼 클릭
driver.find_element(By.CSS_SELECTOR,'#sform > fieldset > button').click()

In [18]:
# 뉴스 탭 클릭
driver.find_element(By.LINK_TEXT,'뉴스').click()

In [6]:
def check_exists_by_selector(driver, selector):
    try:
        driver.find_element(By.CSS_SELECTOR, selector)
    except NoSuchElementException:
        return False
    return True

In [12]:
data_dict = {
            'title' : [],
            'date' : [],
            'reporter' : [],
            'content' : []
}

# 네이버 뉴스 클릭 후 데이터 추출하기
box = driver.find_element(By.CLASS_NAME, 'group_news')

lists = box.find_elements(By.LINK_TEXT,'네이버뉴스')

for li in tqdm(lists):
    news_url = li
    # news_url 을 찾아서 그 위치로 이동
    actions = ActionChains(driver).move_to_element(news_url)
    actions.perform()

    # 클릭시 새로운 탭으로 전환 (window: Keys.CONTROL, Mac: Keys.COMMAND)
    news_url.send_keys(Keys.COMMAND + '\n')
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(3)

    title = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_title').text.strip()
    data_dict['title'].append(title)
    date = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_journalist > div > div.media_end_head_info_datestamp > div > div > div:nth-child(1) > span').text.strip()
    data_dict['date'].append(date)
    if check_exists_by_selector(driver, '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_journalist > div > div.media_end_head_journalist_name_area'):
        reporter = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_journalist > div > div.media_end_head_journalist_name_area').text.strip()
    else:
        reporter = ''
    data_dict['reporter'].append(reporter)
    content = driver.find_element(By.CSS_SELECTOR, '#contents').text.strip()
    data_dict['content'].append(content)

    driver.close()
    driver.switch_to.window(driver.window_handles[0]) 

  0%|          | 0/25 [00:00<?, ?it/s]

In [8]:
# DataFrame 만들기
df = pd.DataFrame(data_dict)

In [ ]:
# csv 저장
df.to_csv(f'navernews_{keyword}.csv')

In [5]:
# 허깅페이스 로그인
import requests
from dotenv import load_dotenv

load_dotenv()

API_TOKEN = os.getenv("HUGGINGFACE_API_TOKEN")

if API_TOKEN:
    print("토큰 로드 성공!")
else:
    print("토큰을 찾을 수 없습니다. .env 파일을 확인하세요.")


토큰 로드 성공!


In [13]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-summarization")
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-summarization")

def summarize_news(text: str) -> str:
    inputs = tokenizer(
        [text],
        max_length=1024,
        truncation=True,
        return_tensors="pt"
    )

    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        max_length=128,
        min_length=20,
        early_stopping=True,
        no_repeat_ngram_size=3
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

article = data_dict['content'][0]
print(summarize_news(article))

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

현행 특감찰관 제도의 대상과 비위행위 범위를 확대하는 '김현지 특별감찰법 개정안'(김현지 특감법)을 발의한 '유상범 국민의힘 의원은 '대통령실 1급 이상 또는 이에 상당하는 공무원'으로 넓히고, '권한을 넘은 영향력 행사'를 명시한 개정안을 오늘(11일) 대표 발의하였다.
